In [1]:
pip install lightfm

     |████████████████████████████████| 310 kB 4.4 MB/s eta 0:00:01
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-macosx_10_9_x86_64.whl size=436455 sha256=982e596dabc4de0212c3bb5b2eb48c2ff48d6e7e02f4725a4864ef74e0853baf
  Stored in directory: /Users/shambhavirege/Library/Caches/pip/wheels/ec/bb/51/9c487d021c1373b691d13cadca0b65b6852627b1f3f43550fa
Successfully built lightfm
Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install scikit-optimize

     |████████████████████████████████| 100 kB 391 kB/s ta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.special import expit
import pickle
import csv
import copy
import itertools
from lightfm import LightFM
import lightfm.evaluation
import sys
sys.path.append('../')

In [5]:
df = pd.read_csv('ml-latest-small/ratings.csv')

In [6]:
def threshold_interactions_df(df, row_name, col_name, row_min, col_min):
    """Limit interactions df to minimum row and column interactions.
    Parameters
    ----------
    df : DataFrame
        DataFrame which contains a single row for each interaction between
        two entities. Typically, the two entities are a user and an item.
    row_name : str
        Name of column in df which corresponds to the eventual row in the
        interactions matrix.
    col_name : str
        Name of column in df which corresponds to the eventual column in the
        interactions matrix.
    row_min : int
        Minimum number of interactions that the row entity has had with
        distinct column entities.
    col_min : int
        Minimum number of interactions that the column entity has had with
        distinct row entities.
    Returns
    -------
    df : DataFrame
        Thresholded version of the input df. Order of rows is not preserved.
    Examples
    --------
    df looks like:
    user_id | item_id
    =================
      1001  |  2002
      1001  |  2004
      1002  |  2002
    thus, row_name = 'user_id', and col_name = 'item_id'
    If we were to set row_min = 2 and col_min = 1, then the returned df would
    look like
    user_id | item_id
    =================
      1001  |  2002
      1001  |  2004
    """

    n_rows = df[row_name].unique().shape[0]
    n_cols = df[col_name].unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_rows*n_cols) * 100
    print('Starting interactions info')
    print('Number of rows: {}'.format(n_rows))
    print('Number of cols: {}'.format(n_cols))
    print('Sparsity: {:4.3f}%'.format(sparsity))

    done = False
    while not done:
        starting_shape = df.shape[0]
        col_counts = df.groupby(row_name)[col_name].count()
        df = df[~df[row_name].isin(col_counts[col_counts < col_min].index.tolist())]
        row_counts = df.groupby(col_name)[row_name].count()
        df = df[~df[col_name].isin(row_counts[row_counts < row_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True

    n_rows = df[row_name].unique().shape[0]
    n_cols = df[col_name].unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_rows*n_cols) * 100
    print('Ending interactions info')
    print('Number of rows: {}'.format(n_rows))
    print('Number of columns: {}'.format(n_cols))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    return df


In [16]:
def df_to_matrix(df, row_name, col_name):
    """Take interactions dataframe and convert to a sparse matrix
    Parameters
    ----------
    df : DataFrame
    row_name : str
    col_name : str
    Returns
    -------
    interactions : sparse csr matrix
    rid_to_idx : dict
    idx_to_rid : dict
    cid_to_idx : dict
    idx_to_cid : dict
    """

    rid_to_idx, idx_to_rid,\
        cid_to_idx, idx_to_cid = get_df_matrix_mappings(df,
                                                        row_name,
                                                        col_name)

    def map_ids(row, mapper):
        return mapper[row]

    I = df[row_name].apply(map_ids, args=[rid_to_idx]).to_numpy()
    J = df[col_name].apply(map_ids, args=[cid_to_idx]).to_numpy()
    V = np.ones(I.shape[0])
    interactions = sp.coo_matrix((V, (I, J)), dtype=np.float64)
    interactions = interactions.tocsr()
    return interactions, rid_to_idx, idx_to_rid, cid_to_idx, idx_to_cid

In [17]:
def get_df_matrix_mappings(df, row_name, col_name):
    """Map entities in interactions df to row and column indices
    Parameters
    ----------
    df : DataFrame
        Interactions DataFrame.
    row_name : str
        Name of column in df which contains row entities.
    col_name : str
        Name of column in df which contains column entities.
    Returns
    -------
    rid_to_idx : dict
        Maps row ID's to the row index in the eventual interactions matrix.
    idx_to_rid : dict
        Reverse of rid_to_idx. Maps row index to row ID.
    cid_to_idx : dict
        Same as rid_to_idx but for column ID's
    idx_to_cid : dict
    """


    # Create mappings
    rid_to_idx = {}
    idx_to_rid = {}
    for (idx, rid) in enumerate(df[row_name].unique().tolist()):
        rid_to_idx[rid] = idx
        idx_to_rid[idx] = rid

    cid_to_idx = {}
    idx_to_cid = {}
    for (idx, cid) in enumerate(df[col_name].unique().tolist()):
        cid_to_idx[cid] = idx
        idx_to_cid[idx] = cid

    return rid_to_idx, idx_to_rid, cid_to_idx, idx_to_cid


In [18]:
df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [19]:
new_df = threshold_interactions_df(df, 'userId', 'movieId', 5, 5)

Starting interactions info
Number of rows: 610
Number of cols: 9724
Sparsity: 1.700%
Ending interactions info
Number of rows: 610
Number of columns: 3650
Sparsity: 4.055%


In [20]:
new_df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100830,610,166528,4.0,1493879365
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047


In [21]:
likes, uid_to_idx, idx_to_uid,mid_to_idx, idx_to_mid = df_to_matrix(new_df, 'userId', 'movieId')

likes

<610x3650 sparse matrix of type '<class 'numpy.float64'>'
	with 90274 stored elements in Compressed Sparse Row format>

In [22]:
def train_test_split(interactions, split_count, fraction=None):
    """
    Split recommendation data into train and test sets
    Params
    ------
    interactions : scipy.sparse matrix
        Interactions between users and items.
    split_count : int
        Number of user-item-interactions per user to move
        from training to test set.
    fractions : float
        Fraction of users to split off some of their
        interactions into test set. If None, then all
        users are considered.
    """
    # Note: likely not the fastest way to do things below.
    train = interactions.copy().tocoo()
    test = sp.lil_matrix(train.shape)

    if fraction:
        try:
            user_index = np.random.choice(
                np.where(np.bincount(train.row) >= split_count * 2)[0],
                replace=False,
                size=np.int64(np.floor(fraction * train.shape[0]))
            ).tolist()
        except:
            print(('Not enough users with > {} '
                  'interactions for fraction of {}')\
                  .format(2*split_count, fraction))
            raise
    else:
        user_index = range(train.shape[0])

    train = train.tolil()

    for user in user_index:
        test_interactions = np.random.choice(interactions.getrow(user).indices,
                                        size=split_count,
                                        replace=False)
        train[user, test_interactions] = 0.
        # These are just 1.0 right now
        test[user, test_interactions] = interactions[user, test_interactions]


    # Test and training are truly disjoint
    assert(train.multiply(test).nnz == 0)
    return train.tocsr(), test.tocsr(), user_index

In [23]:
train, test, user_index = train_test_split(likes, 5, fraction=0.2)

In [24]:
eval_train = train.copy()
non_eval_users = list(set(range(train.shape[0])) - set(user_index))

eval_train = eval_train.tolil()
for u in non_eval_users:
    eval_train[u, :] = 0.0
eval_train = eval_train.tocsr()

In [27]:
feat_dlist = [{} for _ in idx_to_mid]
for idx, row in new_df.iterrows():
    feat_key = '{}'.format(row.rating)
    idx = mid_to_idx.get(row.movieId)
    if idx is not None:
        feat_dlist[idx][feat_key] = 1

In [29]:
from sklearn.feature_extraction import DictVectorizer

In [30]:
dv = DictVectorizer()
item_features = dv.fit_transform(feat_dlist)

In [31]:
item_features

<3650x10 sparse matrix of type '<class 'numpy.float64'>'
	with 21041 stored elements in Compressed Sparse Row format>

In [42]:
import skopt

In [43]:
from skopt import forest_minimize

In [44]:
def objective(params):
    # unpack
    epochs, learning_rate,\
    no_components, alpha = params
    
    user_alpha = alpha
    item_alpha = alpha
    model = LightFM(loss='warp',
                    random_state=2016,
                    learning_rate=learning_rate,
                    no_components=no_components,
                    user_alpha=user_alpha,
                    item_alpha=item_alpha)
    model.fit(train, epochs=epochs,
              num_threads=4, verbose=True)
    
    patks = lightfm.evaluation.precision_at_k(model, test,
                                              train_interactions=None,
                                              k=5, num_threads=4)
    mapatk = np.mean(patks)
    # Make negative because we want to _minimize_ objective
    out = -mapatk
    # Handle some weird numerical shit going on
    if np.abs(out + 1) < 0.01 or out < -1.0:
        return 0.0
    else:
        return out

In [45]:
space = [(1, 260), # epochs
         (10**-4, 1.0, 'log-uniform'), # learning_rate
         (20, 200), # no_components
         (10**-6, 10**-1, 'log-uniform'), # alpha
        ]

res_fm = forest_minimize(objective, space, n_calls=250,
                     random_state=0,
                     verbose=True)

Epoch:   0%|          | 0/48 [00:00<?, ?it/s]

Iteration No: 1 started. Evaluating function at random point.


Epoch:  20%|██        | 2/10 [00:00<00:00, 11.14it/s]

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 7.3832
Function value obtained: -0.0016
Current minimum: -0.0016
Iteration No: 2 started. Evaluating function at random point.


Epoch:   1%|          | 1/89 [00:00<00:16,  5.22it/s]

...

Iteration No: 248 ended. Search finished for the next optimal point.
Time taken: 52.6809
Function value obtained: -0.0377
Current minimum: -0.0525
Iteration No: 249 started. Searching for the next optimal point.


Epoch:   0%|          | 0/241 [00:00<?, ?it/s]

Iteration No: 249 ended. Search finished for the next optimal point.
Time taken: 54.2348
Function value obtained: -0.0393
Current minimum: -0.0525
Iteration No: 250 started. Searching for the next optimal point.


Epoch: 100%|██████████| 241/241 [01:05<00:00,  3.69it/s]


Iteration No: 250 ended. Search finished for the next optimal point.
Time taken: 65.8754
Function value obtained: -0.0361
Current minimum: -0.0525


In [46]:
print('Maximimum p@k found: {:6.5f}'.format(-res_fm.fun))
print('Optimal parameters:')
params = ['epochs', 'learning_rate', 'no_components', 'alpha']
for (p, x_) in zip(params, res_fm.x):
    print('{}: {}'.format(p, x_))

Maximimum p@k found: 0.05246
Optimal parameters:
epochs: 180
learning_rate: 0.07220663065048905
no_components: 49
alpha: 0.0015905757652518876


In [47]:
df = pd.read_csv('ml-latest/ratings.csv')

In [48]:
new_df = threshold_interactions_df(df, 'userId', 'movieId', 5, 5)

Starting interactions info
Number of rows: 283228
Number of cols: 53889
Sparsity: 0.182%
Ending interactions info
Number of rows: 266956
Number of columns: 30808
Sparsity: 0.336%


In [49]:
likes, uid_to_idx, idx_to_uid,mid_to_idx, idx_to_mid = df_to_matrix(new_df, 'userId', 'movieId')

In [50]:
likes

<266956x30808 sparse matrix of type '<class 'numpy.float64'>'
	with 27671427 stored elements in Compressed Sparse Row format>

In [51]:
train, test, user_index = train_test_split(likes, 5, fraction=0.2)

In [52]:
eval_train = train.copy()
non_eval_users = list(set(range(train.shape[0])) - set(user_index))

eval_train = eval_train.tolil()
for u in non_eval_users:
    eval_train[u, :] = 0.0
eval_train = eval_train.tocsr()

In [54]:
feat_dlist = [{} for _ in idx_to_mid]
for idx, row in new_df.iterrows():
    feat_key = '{}'.format(row.rating)
    idx = mid_to_idx.get(row.movieId)
    if idx is not None:
        feat_dlist[idx][feat_key] = 1

In [55]:
dv = DictVectorizer()
item_features = dv.fit_transform(feat_dlist)

In [ ]:
space = [(1, 260), # epochs
         (10**-4, 1.0, 'log-uniform'), # learning_rate
         (20, 200), # no_components
         (10**-6, 10**-1, 'log-uniform'), # alpha
        ]

res_fm = forest_minimize(objective, space, n_calls=250,
                     random_state=0,
                     verbose=True)